<a href="https://colab.research.google.com/github/arkeodev/XAI/blob/main/XAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Explainable AI

Explainable AI (XAI) refers to methods and techniques in the field of artificial intelligence (AI) that make the outputs of machine learning models understandable to humans.

As AI systems are deployed in more sensitive and high-stakes areas such as healthcare, finance, and legal systems, the need for accountability, trust, and fairness in AI decisions has become paramount. Here are some reasons why XAI is becoming more important:

1. Trust and Reliability
2. Compliance and Regulation
3. Debugging and Improvement
4. Ethical Considerations
5. Collaborative Human-AI Decision Making


# The Balance Between Accuracy and Interpretability

<figure>
    <img src="https://raw.githubusercontent.com/arkeodev/XAI/main/images/balance_between_accuracy_and_interprebality.png#:~:text=Copy-,permalink,-%E2%8C%98" width="800" height="500" alt="Balance between accuracy and interpretability of models">
    <figcaption>Balance between accuracy of models and interpretability of models</figcaption>
</figure>

The balance between accuracy and interpretability in AI, as illustrated in the image, highlights a fundamental trade-off:

**complex models** like neural networks achieve high accuracy but are less interpretable, making it challenging to understand their decision-making processes.

**simpler models** like decision trees and linear regression offer greater interpretability, albeit sometimes at the expense of accuracy.

## Black Box vs. White Box Models

### Black Box Models

**Definition:** Models whose internal workings are not transparent or easily interpretable. They are complex, making it difficult to understand how inputs are processed to produce outputs.

**Importance:** While they can achieve high accuracy, especially in complex tasks, their lack of transparency is a challenge for trust, accountability, and compliance in critical applications.

**Examples:**

Deep Neural Networks (DNNs): Used for tasks like image recognition and natural language processing, but their multi-layered structure makes them inherently difficult to interpret.

Ensemble Models (e.g., Random Forests): Aggregate the decisions of multiple models, which increases predictive power but complicates understanding of the decision-making process.

### White Box Models

**Definition:** Models that are inherently transparent, with their internal mechanics easily understandable by humans. The process from input to output is clear, making these models preferable for situations requiring interpretability.

**Importance:** Essential for applications where understanding the decision-making process is as important as the decision itself, such as in healthcare or criminal justice. They facilitate trust, debugging, and compliance.

**Examples:**

Linear Regression: Provides a clear equation where the contribution of each feature is directly understandable.

Decision Trees: Offer a straightforward, rule-based approach to decision-making, where the path from input to output is easily traceable.

# Categorization in XAI

## Agnosticity

- `Model-agnostic`: Applicable to all model types. _These methods can be used with any type of machine learning model without specific tailoring._
- `Model-specific`: Only applicable to a specific model type. _These techniques are designed for particular model architectures._


## Scope

- `Global explanation`: Explaining the whole model. _This provides an overview of how the model makes decisions on a general level._
- `Local explanation`: Explaining individual predictions. _Focuses on explaining the decision-making process for a single instance._

<figure>
    <img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/global_vs_local.png#:~:text=Copy-,permalink,-%E2%8C%98" width="400" height="250" alt="The scope of interpretability">
    <figcaption>Global vs Local Interpretability</figcaption>
</figure>

## Data type

- `Graph`: _Used for models that process data in the form of graphs or networks._
- `Image`: _Applicable for models that work with image data._
- `Text / Speech`: _Covers models designed to understand and generate text or speech._
- `Tabular`: _Refers to models dealing with structured data arranged in tables._


## Explanation type

- `Visual`: _This involves using visual tools to illustrate the model's workings._
- `Data points`: _Highlights specific data points to explain model predictions._
- `Feature importance`: _Shows which features are most influential in the model's predictions._
- `Surrogate models`: _Involves creating a simpler model that approximates the behavior of the complex one._


# Demo of By-design Interpretable White Box Models



> The source code is taken from: https://github.com/deepfindr/xai-series



In [2]:
! pip install interpret -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.7/778.7 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [3]:
# Imports

import pandas as pd
# Makes sure we see all columns
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from imblearn.over_sampling import RandomOverSampler
from interpret.glassbox import (LogisticRegression,
                                ClassificationTree,
                                ExplainableBoostingClassifier)
from interpret import show


In [4]:
# Utils
class DataLoader():
    def __init__(self):
        self.data = None

    def load_dataset(self, path="data/healthcare-dataset-stroke-data.csv"):
        self.data = pd.read_csv(path)

    def preprocess_data(self):
        # One-hot encode all categorical columns
        categorical_cols = ["gender",
                            "ever_married",
                            "work_type",
                            "Residence_type",
                            "smoking_status"]
        encoded = pd.get_dummies(self.data[categorical_cols],
                                prefix=categorical_cols)

        # Update data with new columns
        self.data = pd.concat([encoded, self.data], axis=1)
        self.data.drop(categorical_cols, axis=1, inplace=True)

        # Impute missing values of BMI
        self.data.bmi = self.data.bmi.fillna(0)

        # Drop id as it is not relevant
        self.data.drop(["id"], axis=1, inplace=True)

        # Standardization
        # Usually we would standardize here and convert it back later
        # But for simplification we will not standardize / normalize the features

    def get_data_split(self):
        X = self.data.iloc[:,:-1]
        y = self.data.iloc[:,-1]
        return train_test_split(X, y, test_size=0.20, random_state=2021)

    def oversample(self, X_train, y_train):
        oversample = RandomOverSampler(sampling_strategy='minority')
        # Convert to numpy and oversample
        x_np = X_train.to_numpy()
        y_np = y_train.to_numpy()
        x_np, y_np = oversample.fit_resample(x_np, y_np)
        # Convert back to pandas
        x_over = pd.DataFrame(x_np, columns=X_train.columns)
        y_over = pd.Series(y_np, name=y_train.name)
        return x_over, y_over


In [5]:
# %% Load and preprocess data
data_loader = DataLoader()
data_loader.load_dataset()
data_loader.preprocess_data()

# Split the data for evaluation
X_train, X_test, y_train, y_test = data_loader.get_data_split()
print(X_train.shape)
print(X_test.shape)
# Oversample the train data
X_train, y_train = data_loader.oversample(X_train, y_train)
print("After oversampling:", X_train.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'data/healthcare-dataset-stroke-data.csv'

## Logistic Regression Demo

In [6]:


# %% Fit logistic regression model
lr = LogisticRegression(random_state=2021, feature_names=X_train.columns, penalty='l1', solver='liblinear')
lr.fit(X_train, y_train)
print("Training finished.")

# %% Evaluate logistic regression model
y_pred = lr.predict(X_test)
print(f"F1 Score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")

# %% Explain local prediction
lr_local = lr.explain_local(X_test[:100], y_test[:100], name='Logistic Regression')
show(lr_local)

# %% Explain global logistic regression model
lr_global = lr.explain_global(name='Logistic Regression')
show(lr_global)

# %% Fit decision tree model
tree = ClassificationTree()
tree.fit(X_train, y_train)
print("Training finished.")
y_pred = tree.predict(X_test)
print(f"F1 Score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")

# %% Explain local prediction
tree_local = tree.explain_local(X_test[:100], y_test[:100], name='Tree')
show(tree_local)

# %% Fit Explainable Boosting Machine
ebm = ExplainableBoostingClassifier(random_state=2021)
ebm.fit(X_train, y_train)
print("Training finished.")
y_pred = ebm.predict(X_test)
print(f"F1 Score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")

# %% Explain locally
ebm_local = ebm.explain_local(X_test[:100], y_test[:100], name='EBM')
show(ebm_local)

# %% Explain globally
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)
# %%

(4088, 21)
(1022, 21)
After oversampling: (7778, 21)
Training finished.
F1 Score 0.5219080540220116
Accuracy 0.7436399217221135


Training finished.
F1 Score 0.5259920468974536
Accuracy 0.761252446183953


KeyboardInterrupt: 

# Tools and Techniques for Explaining AI Models

## Local Interpretable Model-Agnostic Explanations (LIME)

Describe LIME and how it offers insights into the predictions of complex, less interpretable models.

## SHapley Additive exPlanations (SHAP)

Discuss SHAP and its role in assessing the contribution of features in highly accurate models.

## Counterfactual Explanations and Adversarial Attacks

Explain how counterfactuals and adversarial examples can highlight model vulnerabilities and enhance interpretability.

## Layerwise Relevance Propagation (LRP)

LRP is a technique used to explain the predictions of complex models by tracing the prediction back through the layers of the network to the input features, thereby providing a visual map or a set of influential features that led to the decision.

[Explainable AI Demos](https://lrpserver.hhi.fraunhofer.de/) is an educational or demonstration tool designed to make AI more accessible and understandable to users by visually and interactively showcasing how AI models arrive at their conclusions. There are four different types of demos:

- Handwriting Classification: This demo seems to use LRP to explain how a neural network trained on the MNIST dataset predicts handwritten digits. It suggests that users can input their handwriting for the AI to classify and explain.
- Image Classification: A more advanced LRP demo for image classification that uses a neural network implemented with Caffe, a deep learning framework. This demo likely illustrates how the AI model determines the content of images.
- Text Classification: This is for classifying natural language documents. The neural network provides predictions on the document's semantic category and uses LRP to explain the classification process.
- Visual Question Answering: This demo allows users to ask AI questions about an image and receive not only answers but also visual explanations that highlight relevant parts of the image involved in the AI's reasoning.

<table>
<tr>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/text_recognition_and_classification.png#:~:text=Copy-,permalink,-%E2%8C%98" alt="Text Recognition and Classification" width="600" /></td>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/image_classification.png#:~:text=Copy-,permalink,-%E2%8C%98" alt="Image Classification" width="600" /></td>
</tr>
<tr>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/mnist_image_recognition.png#:~:text=.-,Copy,-permalink" alt="MNIST Image Recognition" width="600" /></td>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/question_and_answering.png#:~:text=Copy-,permalink,-%E2%8C%98" alt="Question and Answering" width="600" /></td>
</tr>
</table>

# Conclusion

- Summarize the importance of XAI and its implications for the development of ethical and transparent AI systems.

# References and Further Reading

- Interpretable Machine Learning Book: https://christophm.github.io/interpretable-ml-book/
- Wonderful video archive of DeepFndr: https://www.youtube.com/playlist?list=PLV8yxwGOxvvovp-j6ztxhF3QcKXT6vORU
- “Why Should I Trust You? ” Explaining the Predictions of Any Classifier: https://arxiv.org/pdf/1602.04938.pdf
- Explainable AI Demos: https://lrpserver.hhi.fraunhofer.de/
- Source Code of Demo of By-design Interpretable White Box Models: Explainable AI explained! by [DeepFindr](https://www.youtube.com/@DeepFindr): https://github.com/deepfindr/xai-series
- Source Data of Demo of By-design Interpretable White Box Models: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset